In [ ]:
pip install lpips

In [ ]:
!pip install tensorflow-addons

In [ ]:
!pip install elasticdeform

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import sys
import numpy as np
import nibabel as nib
import tensorflow as tf
import os
import pandas as pd
import scipy.io as sio

In [ ]:
sys.path.append('/content/drive/Othercomputers/My Mac/Documents/GitHub/CCL-Synthetis/trained/CCL-Synthetis/')

# Assuming utils and Datagen are your local modules or packages
from utils.model_utils import modelObj
from Synthesis.synthesis_losses import lossObj
from Datagen.h5_pretrain_Synth_Data_Generator import DataLoaderObj

import Synthesis.synth_config as cfg

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [ ]:
# Load the model
# model_ = modelObj(cfg)
#--------------------------------------------
# weights = {
#     'Baseline': '/content/drive/Othercomputers/My Mac/Documents/GitHub/CCL-Synthetis/trained/weights/ReLU/Loss/brats_t1_t2_flair_loss5/rand_deform/finetune_Hybride _baseline/t1_t2_flair_loss5_tr1_Hybride/tr_comb_Exp1_LR_ft_0.01/weights_20Final.hdf5',
#     'Full_Decoder': '/content/drive/Othercomputers/My Mac/Documents/GitHub/CCL-Synthetis/trained/weights/ReLU/Loss/brats_t1_t2_flair_loss5/rand_deform/finetune_Hybride _CL_full_dec_warm/t1_t2_flair_loss5_tr1_Hybride/tr_comb_Exp1_LR_ft_0.01/weights_20Final.hdf5',
#     'Partial_Decoder': '/content/drive/Othercomputers/My Mac/Documents/GitHub/CCL-Synthetis/trained/weights/ReLU/Loss/brats_t1_t2_flair_loss5/rand_deform/finetune_Hybride _CL_partial_dec_warm/t1_t2_flair_loss5_tr1_Hybride/tr_comb_Exp1_LR_ft_0.01/weights_20Final.hdf5'
# }

# model_base = model_.synth_unet(act_name='relu')
# model_full = model_.synth_unet(act_name='relu')
# model_part = model_.synth_unet(act_name='relu')

# # Load weights into models
# model_base.load_weights(weights['Baseline'])
# model_full.load_weights(weights['Full_Decoder'])
# model_part.load_weights(weights['Partial_Decoder'])


In [ ]:
sys.path.append('/gpfs/scratch/pa2297/multi-contrast-contrastive-learning/')

from utils.utils import myCrop3D
from utils.utils import contrastStretch

def normalize_img_zmean(img, mask):
    ''' Zero mean unit standard deviation normalization based on a mask'''
    mask_signal = img[mask>0]
    mean_ = mask_signal.mean()
    std_ = mask_signal.std()
    img = (img - mean_ )/ std_
    return img, mean_, std_

def normalize_img(img):
    img = (img - img.min())/(img.max()-img.min())
    return img

def load_subject(datadir, subName):
    data_suffix = ['_t1ce.nii.gz', '_t2.nii.gz', '_t1.nii.gz', '_flair.nii.gz']
    sub_img = []
    mask = None
    subject_dir = os.path.join(datadir, subName)  # Correctly form the path to the subject's directory

    for suffix in data_suffix:
        img_path = os.path.join(subject_dir, subName + suffix)  # Correct path to the image file
        img_data = nib.load(img_path).get_fdata()
        img_data = np.rot90(img_data, -1)
        img_data = myCrop3D(img_data, (192,192))

        if mask is None:
            mask = np.zeros(img_data.shape)
            mask[img_data > 0] = 1

        img_data = contrastStretch(img_data, mask, 0.01, 99.9)
        img_data, mean_, std_ = normalize_img_zmean(img_data, mask)
        sub_img.append(img_data)

    sub_img = np.stack(sub_img, axis=-1)
    sub_img = np.transpose(sub_img, (2, 0, 1, 3))  # Adjust dimensions as needed
    sub_img = sub_img[50:120]  # Assuming your volume z-axis slice range

    return sub_img

#-----------------------------------------------------------------

def get_data(img, contrast_idx, target_contrast_idx):
    """Returns tuple (input, target) correspond to sample #idx."""
    x_train = generate_X(img, contrast_idx)
    y_train = generate_Y(img, target_contrast_idx)
    return tf.identity(x_train), tf.identity(y_train)

def generate_X(img, contrast_idx):
    X = img[..., contrast_idx]
    return X

def generate_Y(img, target_contrast_idx):
    Y = img[..., target_contrast_idx]
    return Y

In [ ]:
import lpips
import torch

In [ ]:
# loss_fn = lpips.LPIPS(net='alex')  # Using AlexNet
loss_fn = lpips.LPIPS(net='vgg')  # Using VGG

Setting up [LPIPS] perceptual loss: trunk [vgg], v[0.1], spatial [off]


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading model from: /usr/local/lib/python3.10/dist-packages/lpips/weights/v0.1/vgg.pth


In [ ]:
def load_predictions(predictions_file):
    data = sio.loadmat(predictions_file)
    # Assuming the data structure is known and 'predictions' is the key
    predictions = {
        'Baseline': data['predictions']['Baseline'][0, 0],
        'Partial_Decoder': data['predictions']['Partial_Decoder'][0, 0],
        'Full_Decoder': data['predictions']['Full_Decoder'][0, 0]
    }
    # Decode byte arrays if necessary
    for key, value in predictions.items():
        if isinstance(value, bytes):
            # Example of decoding bytes to numpy array; adapt as needed
            predictions[key] = np.frombuffer(value, dtype=np.float32).reshape((80, 192, 192, 3))
    return predictions

In [ ]:
import tensorflow as tf
import torch

def prepare_image(image):
    # Ensure the input image is a TensorFlow tensor with dtype float32
    image = tf.convert_to_tensor(image, dtype=tf.float32)

    # Normalize the image tensor to [-1, 1]
    image_min = tf.reduce_min(image)
    image_max = tf.reduce_max(image)
    image = 2 * (image - image_min) / (image_max - image_min) - 1

    # Check and adapt the tensor dimensions
    # Assumption: The last dimension is channels if it's exactly 3; otherwise, we assume 1 channel.
    if image.shape[-1] != 3:
        # If the image does not have three channels, we tile to create three channels
        image = tf.tile(image, [1, 1, 3])  # This expects image to be at least 3D; reshape if not

    # Ensure the image tensor is in the correct shape (channels, height, width)
    image = tf.transpose(image, [2, 0, 1])

    # Convert TensorFlow tensor to a PyTorch tensor
    image = torch.from_numpy(image.numpy()).float()

    # Ensure the image tensor has a batch dimension
    if image.dim() == 3:
        image = image.unsqueeze(0)

    return image



In [ ]:
def compute_lpips(datadir, predictions_dir, cfg):
    # loss_fn = lpips.LPIPS(net='alex')  # Initialize the LPIPS function
    subject_scores = []
    subjects = sorted([f for f in os.listdir(datadir) if f.startswith("BraTS2021_")])

    for subName in subjects:
        print(f"Processing {subName}...")
        img = load_subject(datadir, subName)
        _, y_true = get_data(img, cfg.contrast_idx, cfg.target_contrast_idx)  # Ground truth images
        y_true = y_true.numpy()

        # Load predictions
        predictions_file = f'{predictions_dir}/{subName}_predictions.mat'
        predictions = load_predictions(predictions_file)

        model_scores = {}
        for model_name, y_pred in predictions.items():
            scores = []
            for i in range(y_true.shape[0]):  # Assuming y_true is numpy array
                img0 = prepare_image(y_true[i])
                # print(type(y_pred))
                img1 = prepare_image(y_pred[i])
                with torch.no_grad():
                    score = loss_fn(img0, img1)
                scores.append(score.item())
            model_scores[model_name] = np.mean(scores)

        subject_scores.append({
            'Subject_ID': subName,
            'Baseline_Avg_LPIPS': model_scores['Baseline'],
            'Full_Decoder_Avg_LPIPS': model_scores['Full_Decoder'],
            'Partial_Decoder_Avg_LPIPS': model_scores['Partial_Decoder']
        })

    return pd.DataFrame(subject_scores)

In [ ]:
datadir = '/content/drive/Othercomputers/My Mac/Documents/GitHub/CCL-Synthetis/trained/BraTS2021_Test/'
predictions_dir = '/content/drive/Othercomputers/My Mac/Documents/GitHub/CCL-Synthetis/trained/predictions/updated_loss5/'

results_df = compute_lpips(datadir, predictions_dir, cfg)
results_df.to_csv('/content/drive/Othercomputers/My Mac/Documents/Vgg_updated.csv', index=False)
print('CSV file has been saved.')

Processing BraTS2021_00016...
Processing BraTS2021_00488...
Processing BraTS2021_00491...
Processing BraTS2021_00494...
Processing BraTS2021_00505...
Processing BraTS2021_00510...
Processing BraTS2021_00516...
Processing BraTS2021_00533...
Processing BraTS2021_00610...
Processing BraTS2021_00618...
Processing BraTS2021_00636...
Processing BraTS2021_00674...
Processing BraTS2021_00765...
Processing BraTS2021_01100...
Processing BraTS2021_01102...
Processing BraTS2021_01105...
Processing BraTS2021_01106...
Processing BraTS2021_01112...
Processing BraTS2021_01113...
Processing BraTS2021_01114...
Processing BraTS2021_01115...
Processing BraTS2021_01117...
Processing BraTS2021_01118...
Processing BraTS2021_01120...
Processing BraTS2021_01121...
Processing BraTS2021_01124...
Processing BraTS2021_01125...
Processing BraTS2021_01128...
Processing BraTS2021_01133...
Processing BraTS2021_01137...
Processing BraTS2021_01138...
Processing BraTS2021_01150...
Processing BraTS2021_01153...
Processing

In [ ]:
print(results_df)

         Subject_ID  Baseline_Avg_LPIPS  Full_Decoder_Avg_LPIPS  \
0   BraTS2021_00016            0.108612                0.092257   
1   BraTS2021_00488            0.123527                0.100889   
2   BraTS2021_00491            0.143819                0.133805   
3   BraTS2021_00494            0.122170                0.099736   
4   BraTS2021_00505            0.152652                0.142963   
..              ...                 ...                     ...   
64  BraTS2021_01403            0.163133                0.128086   
65  BraTS2021_01500            0.149235                0.113174   
66  BraTS2021_01516            0.165236                0.145060   
67  BraTS2021_01583            0.130443                0.114228   
68  BraTS2021_01654            0.165639                0.128517   

    Partial_Decoder_Avg_LPIPS  
0                    0.102396  
1                    0.103158  
2                    0.128996  
3                    0.110969  
4                    0.139636  
.. 